In [1]:
import sys
sys.path.append('..')
print(sys.path)

['/home/vinnylg/projects/covid19datascience/notebooks', '/home/vinnylg/.vscode/extensions/ms-toolsai.jupyter-2021.5.745244803/pythonFiles', '/home/vinnylg/.vscode/extensions/ms-toolsai.jupyter-2021.5.745244803/pythonFiles/lib/python', '/home/vinnylg/anaconda3/lib/python38.zip', '/home/vinnylg/anaconda3/lib/python3.8', '/home/vinnylg/anaconda3/lib/python3.8/lib-dynload', '', '/home/vinnylg/.local/lib/python3.8/site-packages', '/home/vinnylg/anaconda3/lib/python3.8/site-packages', '/home/vinnylg/.local/lib/python3.8/site-packages/IPython/extensions', '/home/vinnylg/.ipython', '..']


In [2]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta
from hashlib import sha256
import codecs


from bulletin.tb_pacientes import TbPacientes
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.utils.utils import Timer, auto_fit_columns
from bulletin.utils import static
from bulletin.utils.normalize import *

pd.set_option('display.max_columns', None)

output = join("output","correcoes","casos_confirmados")

if not isdir(output):
    makedirs(output)

timer = Timer()

ModuleNotFoundError: No module named 'unidecode'

In [2]:
timer.reset()
casos_confirmados = CasosConfirmados()
casos_confirmados.update()
timer.stop()

Casos confirmados excluidos: 8
Obitos confirmados excluidos: 1


601.4204819202423

In [3]:
# casos = casos_confirmados.fix_mun_resid_casos('mun_atend')
# obitos = casos_confirmados.fix_mun_resid_obitos()
casos = casos_confirmados.get_casos()
casos = casos.drop(columns=['obito','data_obito','excluir'])
obitos = casos_confirmados.get_obitos()

print(f"{len(casos)};{len(obitos)}")
obitos.loc[~obitos['hash'].isin(casos['hash'])]

633553;11467


,ordem,ibge7,nome,sexo,idade,municipio,rs,data_do_obito,excluir,hash,hash_less,hash_more,hash_dt_obito


In [4]:
obitos.loc[obitos['hash'].isin(casos['hash_less']),'idade'] += 1
obitos.loc[obitos['hash'].isin(casos['hash_less']),'hash'] = obitos.loc[obitos['hash'].isin(casos['hash_less'])].apply(lambda row: normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['municipio']), axis=1)

In [5]:
obitos.loc[obitos['hash'].isin(casos['hash_more']),'idade'] -= 1
obitos.loc[obitos['hash'].isin(casos['hash_more']),'hash'] = obitos.loc[obitos['hash'].isin(casos['hash_more'])].apply(lambda row: normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['municipio']), axis=1)

In [6]:
obitos.loc[~obitos['hash'].isin(casos['hash'])]

,ordem,ibge7,nome,sexo,idade,municipio,rs,data_do_obito,excluir,hash,hash_less,hash_more,hash_dt_obito


In [7]:
obitos = obitos.rename(columns={'data_do_obito': 'data_obito'})
obitos['obito'] = 'SIM'

casos = pd.merge(left=casos, right=obitos[['obito','data_obito','hash']], on='hash', how='left')

casos.loc[casos['obito'].isnull(), 'obito'] = 'NAO'
casos.loc[(casos['obito']=="SIM") & (casos.duplicated('hash')), 'obito'] = 'NAO'
casos.loc[(casos['obito']=="SIM") & (casos.duplicated('hash')), 'data_obito'] = pd.NaT

print(f"{len(casos)};{len(casos.loc[casos['obito']=='SIM'])}")

633553;11467


In [14]:
obitos['excluir'] = None
casos['excluir'] = None

In [15]:
new_casos = casos[['ordem','identificacao','ibge7','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','obito','data_obito','excluir']]
new_obitos = obitos[['ordem','ibge7','nome','sexo','idade','municipio','rs','data_obito','excluir']]
new_obitos.shape

(11467, 9)

In [18]:
writer = pd.ExcelWriter(join(output,"Casos confirmados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

new_casos.to_excel(writer,sheet_name='Casos confirmados',index=None)
new_obitos.to_excel(writer,sheet_name='Obitos',index=None)

writer.save()
writer.close()

In [17]:
writer = pd.ExcelWriter(join(output,f"Casos Confirmados Regionais.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

for grupo_casos, grupo_obitos in zip(new_casos.groupby('rs'),new_obitos.groupby('rs')):
    grupo_casos[1].to_excel(writer,sheet_name=f"casos_rs{grupo_casos[0]}",index=None)
    grupo_obitos[1].to_excel(writer,sheet_name=f"obitos_rs{grupo_casos[0]}",index=None)
    worksheet = writer.sheets[f"casos_rs{grupo_casos[0]}"]
    auto_fit_columns(worksheet,grupo_casos[1])
    worksheet = writer.sheets[f"obitos_rs{grupo_casos[0]}"]
    auto_fit_columns(worksheet,grupo_obitos[1])

new_casos.loc[new_casos['rs'].isnull()].to_excel(writer,sheet_name='casos_fora',index=None)
new_obitos.loc[new_obitos['rs'].isnull()].to_excel(writer,sheet_name='obitos_fora',index=None)

writer.save()
writer.close()